# Introduction

Copyright

Distribution

In order to use the code the following libraries must be installed:
1. Pandas

In [24]:
import pandas as pd
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector
import dwavebinarycsp

# Input and Preprocessing

In this section all data input is entered, validated and preprocessed in order to meet further requirements. 

### Input data file

In this section the following input is proceeded:
1. Input data file with information on ferry connections with
    - From: connection start harbour
    - To: connection end harbour
    - Distance: shortest direct distance between departure and arrival harbours

In [20]:
df_harbours = pd.read_csv("data_harbours_utf8.csv", sep = ";")
df_harbours

,Column1,Bremerhaven,Brunsbüttel,Emden,Hamburg,Kiel,Lübeck,Rostock,Sassnitz,Stralsund,...,St. Petersburg,Gothenburg,Bergen,Oslo,Stavanger,Aberdeen,Immingham,London,Calais,Le Havre
0,Bremerhaven,0.0,NaN,137.0,NaN,135.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,435.0,NaN,NaN,NaN,NaN
1,Brunsbüttel,81.0,0.0,NaN,36.0,54.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Emden,137.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hamburg,117.0,36.0,NaN,0.0,90.0,187.0,174.0,145.0,NaN,...,868.0,NaN,NaN,NaN,408.0,481.0,376.0,428.0,NaN,NaN
4,Kiel,135.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Lübeck,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,93.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Rostock,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Sassnitz,NaN,NaN,NaN,NaN,145.0,NaN,92.0,0.0,NaN,...,661.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Stralsund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Wilhelmshaven,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Formatting of connection data

In order to formulate the problem in a format accessible by the D-Wave libraries the matrix data is restructured to contain the information in an unstacked format. 

In [21]:
# rename 'Column1' to 'From'
df_harbours=df_harbours.rename(columns={'Column1':"From"})

# transform end harbours in columns to rows
df_connections= pd.melt(df_harbours,id_vars=['From'],var_name="To",value_name='Distance')

# filter NaN-values
df_connections=df_connections[df_connections["Distance"].notnull()]

# filter out 0.0 values
df_connections=df_connections[df_connections.Distance != 0]

#reset index
df_connections=df_connections.reset_index(drop=True)
df_connections

,From,To,Distance
0,Brunsbüttel,Bremerhaven,81.0
1,Emden,Bremerhaven,137.0
2,Hamburg,Bremerhaven,117.0
3,Kiel,Bremerhaven,135.0
4,Wismar,Bremerhaven,585.0
...,...,...,...
78,Hamburg,Immingham,376.0
79,Hamburg,London,428.0
80,Aberdeen,London,431.0
81,Rotterdam,Calais,129.0


### User input and validation of departure and destination harbour 

In this section the manual input is requested and validated by the following criteria:
1. departure and destination harbour are part of the input 

If input is invalid, the according error message is displayed.

In [22]:
#Check whether the departure harbour is element of the input data
while True:
    #Entering departure harbour
    departure=input(str("Please enter departure harbour:"))
    if not departure in df_connections['From'].values:
        print("Entered harbour is not defined as a depature harbour in the input file. Please enter valid departure harbour.")
        continue
    else:
        break

#Check whether the destination harbour is element of the input data
while True:
    #Entering destination harbour
    destination=input(str("Please enter destination harbour:"))
    if not destination in df_connections['To'].values:
        print("Entered harbour is not defined as a destination harbour in the input file. Please enter destination valid harbour.")
        continue
    if destination == departure:
        print("Entered destination harbour equal to departure harbour. Please enter valid destination harbour.")
        continue
    else:
        #we're happy with the value given.
        #we're ready to exit the loop.
        break


Entered harbour is not defined as a destination harbour in the input file. Please enter destination valid harbour.
Entered harbour is not defined as a destination harbour in the input file. Please enter destination valid harbour.


### Data enhancement of connection data

A problem can be formulated in different ways in order to provide the quantum computer with it. In this example, the problem is formulated as a binary quadratic model (BQM) in the form of a QUBO. The mathematical phrase can be attained by a support class called Constraint Satisfaction Problem (CSP) or can directly be input. In this case, the problem will first be represented by a CSP. The CSP constitutes of 
1. all variables and 
2. constraints on the variables. 

As the variables in this example will represent the connections between two harbours a set of support variables is needed in order to represent the entered departure and destination harbour. Therefor, artificial connections from "start" harbour to each real harbour and to "end" harbour from each real harbour are created in the following code. 


In [23]:
df_starts = pd.DataFrame({'From':'start', 'To':df_connections['From'].unique(),'Distance':0})
df_ends = pd.DataFrame({'From':df_connections['To'].unique(), 'To':'end','Distance':0})
df_connections= pd.concat([df_starts,df_connections,df_ends])
df_connections

,From,To,Distance
0,start,Brunsbüttel,0.0
1,start,Emden,0.0
2,start,Hamburg,0.0
3,start,Kiel,0.0
4,start,Wismar,0.0
...,...,...,...
30,Aberdeen,end,0.0
31,Immingham,end,0.0
32,London,end,0.0
33,Calais,end,0.0


# Data Preparation for Quantum Computing (Definition of Classes)

In this section, all methods are defined that are later used to
1. create Contraint Satisfaction Problem (CSP)
2. formulate Binary Quadratic Model (BQM) as QUBO. 

In [ ]:
class Ferry:
    def __init__(self):
        self.routes = [
        ["Hamburg"  , "Helgoland", 20],
        ["Hamburg"  , "Romo", 60],
        ["Romo"     , "Esberg", 20],
        ["Helgoland", "Esberg", 10],
        #["Hamburg"  , "London", 10],
        #["Start" , "Hamburg",0],
        #["Esberg", "Ende",0]
        ]
        self.csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)
        self.ports = set([])
        self.n=0
        for route in self.routes:
            self.ports.add(route[0])
            self.ports.add(route[1])

    def get_label(self,start, end, length):
        return "{start},{end}{length}".format(**locals())

    def sum_to_two_or_zero(self,*args):
        fromList=[]
        toList=[]
        x = 0
        print(args)
        for a in args:
            if x != self.n:
                toList.append(a)
                x=x+1
            else:
                fromList.append(a)

        sum_value0 = sum(fromList)
        sum_value1 = sum(toList)
        return ((sum_value0 == 1 and sum_value1 == 1) or (sum_value0 == 0 and sum_value1 == 0))

    def get_bqm(self):
        print(self.ports)
        for port in self.ports:
            directions = []
            self.n=0
            for route in self.routes:
                if (route[0]==port):
                    directions.append("to"+self.get_label(route[0],route[1],route[2]))
                self.n = self.n+1
            if (port=="Hamburg"):
                self.n = self.n+1
                directions.append("toStart,Hamburg0")
            if (port=="Esberg"):
                directions.append("fromEnde,Esberg0")
            for route in self.routes:
                if (route[1]==port):
                    directions.append("from"+self.get_label(route[1],route[0],route[2])) 
            self.csp.add_constraint(self.sum_to_two_or_zero, directions)
        print(self.csp.constraints)
        self.csp.add_variable("toStart,Hamburg0")
        self.csp.add_variable
        self.csp.fix_variable("toStart,Hamburg0",1)
        self.csp.fix_variable("fromEnde,Esberg0",1)
        bqm = dwavebinarycsp.stitch(self.csp)
        return bqm

# Creation of BQM and Transfer to Quantum Sampler